In [44]:
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

In [2]:
import urllib.parse
from bs4 import BeautifulSoup

In [3]:
import requests
import brotli

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [5]:
filein_path = "../../../data/data_wikis/1 - politician data for test scraping/"

In [6]:
fileout_path = "../../../data/data_wikis/2.1 - sk namuwiki scraping test/"

In [48]:
logfile_name = "_seleniumtest_log.txt"

# import sk & tw politician test data

In [7]:
sktest = pd.read_excel(filein_path+"sk_testset.xlsx")

In [8]:
sktest.shape

(10114, 24)

In [9]:
# sktest["URL_NamuWiki"]=np.nan

In [10]:
sktest[100:200]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,CandidateStatus,WonElection,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki
100,강선광,姜善光,19610129,남,53,구·시·군의회의원선거,20140604,Legislator,충청남도,천안시서북구,...,등록,0,NaN,NaN,NaN,정당인,호서대학교 경영학과 졸업,(전)쌍용1동 주민자치위원장,(현)충청남도바르게살기협의회 부회장,NaN
101,강선구,姜先求,19620813,남,51,구·시·군의 장선거,20140604,Chief,인천광역시,중구,...,등록,0,NaN,NaN,NaN,정당인,경희대학교 국어국문학과 졸업,(전)민주당 중구·동구·옹진군 지역위원장,(전)국회정책연구위원,https://namu.wiki/w/%EA%B0%95%EC%84%A0%EA%B5%AC
102,강선구,姜善求,19490218,남,65,구·시·군의회의원선거,20140604,Legislator,대전광역시,서구,...,등록,0,NaN,NaN,NaN,무직,미기재,(전)둔산상포장의사대표,(전)월평지역의용소방대장,https://namu.wiki/w/%EA%B0%95%EC%84%A0%EA%B5%AC
103,강선우,姜仙祐,19780602,여,37,비례대표국회의원선거,20160413,Party List Legislator,전국,전국,...,등록,0,NaN,NaN,NaN,교수,미국 위스콘신대 가족학 박사(6년),(현)사우스다코타주립대 교수,(현)전미 노인학회 정회원,NaN
104,강선화,姜善花,19800818,여,33,시·도의회의원선거,20140604,Legislator,서울특별시,영등포구,...,등록,0,NaN,NaN,NaN,정당인,전북대학교 심리학과 졸업,(전)민주노총 금속노조 선전부장,(현)영등포 뉴타운폐지와 마을공동체 추진포럼 운영위원,NaN
105,강성균,姜性均,19520223,남,62,교육의원선거,20140604,Education Council Member,제주특별자치도,제주시,...,등록,1,44340.0,57.37,NaN,교육활동,제주대학교 교육대학원(석사) 교육학과 국어교육전공 취득,(전) 제주과학고등학교 교장,(전) 탐라교육원장,NaN
106,강성길,姜成吉,19630823,남,50,구·시·군의회의원선거,20140604,Legislator,서울특별시,서초구,...,등록,0,NaN,NaN,NaN,제6대 서초구의원,중앙대학교 사회개발 대학원 사회복지학과 석사과정 재학,(현)제6대 서초구의회 행정복지위원장,(현)세금바로쓰기 납세자 운동 서울시지부 회장,NaN
107,강성길,姜誠吉,19570808,남,56,구·시·군의회의원선거,20140604,Legislator,대구광역시,동구,...,등록,0,NaN,NaN,NaN,직장인,중앙대학교 사회개발대학원 보건행정학과 석사 수료,(전)새정치민주연합 중앙당 정책당원,(현)민주노총 및 공공운수노조 대의원,NaN
108,강성민,姜成旼,19710806,남,42,시·도의회의원선거,20140604,Legislator,제주특별자치도,제주시,...,등록,0,NaN,NaN,NaN,정치인,제주대학교 법정대학 행정학과 졸업,(현)제주미래비전연구원 제주생활정책포럼 대표,(전)제주특별자치도의회 정책자문위원,https://namu.wiki/w/%EA%B0%95%EC%84%B1%EB%AF%BC
109,강성봉,姜聖峰,19710201,남,43,광역의원비례대표선거,20140604,Party List Legislator,전라북도,NaN,...,등록,0,NaN,NaN,NaN,회사원,고려대학교 행정대학원 졸업,(현)새정치민주연합전국청년위원회부위원장,NaN,NaN


In [11]:
# percent duplicates

1 - (sktest["Name_Native"].drop_duplicates().shape[0] / sktest.shape[0])

0.13881748071979438

# scrape namuwiki with selenium

In [57]:
url_stem = "https://namu.wiki"

p_error_name = "data-v-1ac09f2e"
p_error_text = "해당 문서를 찾을 수 없습니다."

In [ ]:
# instantiate selenium webdriver
service = Service(executable_path=ChromeDriverManager().install())

# scrape namuwiki using selenium
for index,p in sktest.iterrows():
    
    if(pd.isnull(p.URL_NamuWiki)):
    
        # construct url
        url= url_stem+"/w/"+urllib.parse.quote(p.Name_Native.encode("utf-8"))

        # reset content & has_content_flag
        has_content_flag = True
        content=""

        try:

            # request url using selenium webdriver
            driver = webdriver.Chrome(service=service)
            driver.get(url)

            # check if page has content
            elements = driver.find_elements(By.TAG_NAME, 'p')

            for e in elements:
                if(e.text==p_error_text):
                    has_content_flag = False

            if(has_content_flag):
                content = driver.page_source

            # close webdriver
            driver.quit()

            # save content
            if(has_content_flag):

                # save output as an html file
                fout = open(fileout_path+"seleniumtest_"+str(index)+"_"+p.Name_Native+".html","w",encoding="utf-8")
                fout.write(content)
                fout.close()

                # update HasNamuWiki content field in dataframe
                #sktest.loc[sktest.Name_Native == p.Name_Native,'URL_NamuWiki']=url

                # save result to log file
                log_output = ",".join([str(index),p.Name_Native,url,"\n"])
                fout = open(fileout_path+logfile_name,"a",encoding="utf-8")
                fout.write(log_output)
                fout.close()

                print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Has Content")


            else:

                # update HasNamuWiki content field in dataframe
                # sktest.loc[sktest.Name_Native == p.Name_Native,'URL_NamuWiki']=False

                # save result to log file
                log_output = ",".join([str(index),p.Name_Native,"False","\n"])
                fout = open(fileout_path+logfile_name,"a",encoding="utf-8")
                fout.write(log_output)
                fout.close()

                print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Empty")

            error_count = 0
            
        except KeyboardInterrupt:
        
            print("Keyboard Interrupt")
            
            break
            

        except Exception as e:

            print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Error")
            print(e)





61 강민국 https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD Number of <p> elements: 5 Error
Message: unknown error: cannot determine loading status
from unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00676463+2188387]
	Ordinal0 [0x0060E461+1762401]
	Ordinal0 [0x00523D78+802168]
	Ordinal0 [0x00517210+750096]
	Ordinal0 [0x0051675A+747354]
	Ordinal0 [0x00515D3F+744767]
	Ordinal0 [0x00514C28+740392]
	Ordinal0 [0x00515228+741928]
	Ordinal0 [0x0051F153+782675]
	Ordinal0 [0x00529FBB+827323]
	Ordinal0 [0x0052D310+840464]
	Ordinal0 [0x005154F6+742646]
	Ordinal0 [0x00529BF3+826355]
	Ordinal0 [0x0057CF6D+1167213]
	Ordinal0 [0x0056C5F6+1099254]
	Ordinal0 [0x00546BE0+945120]
	Ordinal0 [0x00547AD6+948950]
	GetHandleVerifier [0x009171F2+2712546]
	GetHandleVerifier [0x0090886D+2652765]
	GetHandleVerifier [0x0070002A+520730]
	GetHandleVerifier [0x006FEE06+516086]
	Ordinal0 [0x0061468B+1787531]
	Ordinal0 [0x00618E88+1805960]
	Ordinal0 [

184 강원호 https://namu.wiki/w/%EA%B0%95%EC%9B%90%ED%98%B8 Number of <p> elements: 5 Empty
186 강윤희 https://namu.wiki/w/%EA%B0%95%EC%9C%A4%ED%9D%AC Number of <p> elements: 5 Empty
189 강은주 https://namu.wiki/w/%EA%B0%95%EC%9D%80%EC%A3%BC Number of <p> elements: 5 Empty
190 강을석 https://namu.wiki/w/%EA%B0%95%EC%9D%84%EC%84%9D Number of <p> elements: 5 Empty
191 강익자 https://namu.wiki/w/%EA%B0%95%EC%9D%B5%EC%9E%90 Number of <p> elements: 5 Empty
194 강인술 https://namu.wiki/w/%EA%B0%95%EC%9D%B8%EC%88%A0 Number of <p> elements: 5 Empty
195 강장순 https://namu.wiki/w/%EA%B0%95%EC%9E%A5%EC%88%9C Number of <p> elements: 5 Empty
196 강재석 https://namu.wiki/w/%EA%B0%95%EC%9E%AC%EC%84%9D Number of <p> elements: 5 Empty
197 강재헌 https://namu.wiki/w/%EA%B0%95%EC%9E%AC%ED%97%8C Number of <p> elements: 5 Empty
198 강전배 https://namu.wiki/w/%EA%B0%95%EC%A0%84%EB%B0%B0 Number of <p> elements: 5 Empty
199 강정관 https://namu.wiki/w/%EA%B0%95%EC%A0%95%EA%B4%80 Number of <p> elements: 5 Empty
202 강정규 https://namu.wiki/w/%EA%

In [ ]:
# things to check

# 0. drop duplicates. some duplicate pages are different (at least size differs). why?  

# 1. recorded links are valid (no false positives); non-recorded links are actually empty (no false negatives)

# 2. disambiguation? what about names that link to more than one person?
#       - disambiguation links are usually included in the page
#       - e.g. https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD - 강민국
#       - 정치인에 대한 내용은 강민국(정치인) 문서를 참고하십시오.
#       - https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD(%EC%A0%95%EC%B9%98%EC%9D%B8) - 강민국(정치인)

# update sktest with selenium log from latest run

In [26]:
log_columns = ["index","Name_Native","url","dropme"]
log = pd.read_csv(fileout_path+logfile_name,header=None,encoding="utf-8",names=log_columns)
log.drop(columns=["index","dropme"],inplace=True)
log

In [30]:
sktest.shape

(10114, 24)

In [29]:
sktest_updated = sktest.merge(log,how="left",on="Name_Native")
sktest_updated.shape

In [36]:
# update URL_NamuWiki with log results
sktest_updated.loc[sktest_updated.URL_NamuWiki.isnull() & sktest_updated.url.notnull(),"URL_NamuWiki"] = sktest_updated.loc[sktest_updated.URL_NamuWiki.isnull() & sktest_updated.url.notnull(),"url"]

# confirm that changes have been made
sktest_updated.loc[sktest_updated.url.notnull(),["URL_NamuWiki","url"]]

In [50]:
# change "False" strings to boolean False
sktest_updated.loc[sktest_updated.URL_NamuWiki=="False","URL_NamuWiki"]=False
sktest_updated[0:100]

In [41]:
# drop log column (url)
sktest_updated.drop(columns=["url"],inplace=True)
sktest_updated[100:200]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,CandidateStatus,WonElection,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki
100,강선광,姜善光,19610129,남,53,구·시·군의회의원선거,20140604,Legislator,충청남도,천안시서북구,...,등록,0,NaN,NaN,NaN,정당인,호서대학교 경영학과 졸업,(전)쌍용1동 주민자치위원장,(현)충청남도바르게살기협의회 부회장,NaN
101,강선구,姜先求,19620813,남,51,구·시·군의 장선거,20140604,Chief,인천광역시,중구,...,등록,0,NaN,NaN,NaN,정당인,경희대학교 국어국문학과 졸업,(전)민주당 중구·동구·옹진군 지역위원장,(전)국회정책연구위원,https://namu.wiki/w/%EA%B0%95%EC%84%A0%EA%B5%AC
102,강선구,姜善求,19490218,남,65,구·시·군의회의원선거,20140604,Legislator,대전광역시,서구,...,등록,0,NaN,NaN,NaN,무직,미기재,(전)둔산상포장의사대표,(전)월평지역의용소방대장,https://namu.wiki/w/%EA%B0%95%EC%84%A0%EA%B5%AC
103,강선우,姜仙祐,19780602,여,37,비례대표국회의원선거,20160413,Party List Legislator,전국,전국,...,등록,0,NaN,NaN,NaN,교수,미국 위스콘신대 가족학 박사(6년),(현)사우스다코타주립대 교수,(현)전미 노인학회 정회원,NaN
104,강선화,姜善花,19800818,여,33,시·도의회의원선거,20140604,Legislator,서울특별시,영등포구,...,등록,0,NaN,NaN,NaN,정당인,전북대학교 심리학과 졸업,(전)민주노총 금속노조 선전부장,(현)영등포 뉴타운폐지와 마을공동체 추진포럼 운영위원,NaN
105,강성균,姜性均,19520223,남,62,교육의원선거,20140604,Education Council Member,제주특별자치도,제주시,...,등록,1,44340.0,57.37,NaN,교육활동,제주대학교 교육대학원(석사) 교육학과 국어교육전공 취득,(전) 제주과학고등학교 교장,(전) 탐라교육원장,NaN
106,강성길,姜成吉,19630823,남,50,구·시·군의회의원선거,20140604,Legislator,서울특별시,서초구,...,등록,0,NaN,NaN,NaN,제6대 서초구의원,중앙대학교 사회개발 대학원 사회복지학과 석사과정 재학,(현)제6대 서초구의회 행정복지위원장,(현)세금바로쓰기 납세자 운동 서울시지부 회장,NaN
107,강성길,姜誠吉,19570808,남,56,구·시·군의회의원선거,20140604,Legislator,대구광역시,동구,...,등록,0,NaN,NaN,NaN,직장인,중앙대학교 사회개발대학원 보건행정학과 석사 수료,(전)새정치민주연합 중앙당 정책당원,(현)민주노총 및 공공운수노조 대의원,NaN
108,강성민,姜成旼,19710806,남,42,시·도의회의원선거,20140604,Legislator,제주특별자치도,제주시,...,등록,0,NaN,NaN,NaN,정치인,제주대학교 법정대학 행정학과 졸업,(현)제주미래비전연구원 제주생활정책포럼 대표,(전)제주특별자치도의회 정책자문위원,https://namu.wiki/w/%EA%B0%95%EC%84%B1%EB%AF%BC
109,강성봉,姜聖峰,19710201,남,43,광역의원비례대표선거,20140604,Party List Legislator,전라북도,NaN,...,등록,0,NaN,NaN,NaN,회사원,고려대학교 행정대학원 졸업,(현)새정치민주연합전국청년위원회부위원장,NaN,NaN


In [56]:
# refresh and save updated sktest
sktest = sktest_updated
sktest.to_excel(filein_path+"sk_testset.xlsx",index=False)

In [45]:
# delete log file
os.remove(fileout_path+logfile_name)

In [33]:
# DEPRECATED - assign URL_NamuWiki link based on files already scraped - I needed this before I created a log file

# import os
# filelist = os.listdir(fileout_path)
# test = sktest
# url_stem = "https://namu.wiki"
# test[100:200]
# scraped = [f.split("_")[2].split(".")[0].strip() for f in filelist]
# for s in scraped:
#     url = url_stem+"/w/"+urllib.parse.quote(s.encode("utf-8"))
#     # print(url_stem+"/w/"+urllib.parse.quote(s.encode("utf-8")))
#     test.loc[test.Name_Native==s,"URL_NamuWiki"]=url
# test[100:200]